# Adding transition probabilities to random walks

This notebook creates transition probability matrices based on user movement between pages for a *1)* directed graph, and an *2)* undirected graph. Performs repeated random walks using the probability matrices for both the directed and undirected graph, and saves both outputs as a csv file. 

In [6]:
def random_walk(A, G, steps, seed, p=False):
    '''
    A is an adjacency matrix, or a transition probability matrix. These should be CSR sparse matrices.
    Set p=True if using a transition probability matrix.
    G is a networkx graph.
    steps is the number of steps to take in the random walk.
    seed is a page slug for your starting node in the random walk. E.g. "/set-up-business" 
    
    returns a numpy array of node ids visited during the random walk.
    can return numpy array of nodes with their data if nodeData == True
    '''

    # set a seed node
    foundSeed = False
    for current_node_index, node in enumerate(G.nodes(data=True)):
        if node[1]["properties"]["name"] == seed:
            foundSeed = True
            break
    
    if not foundSeed:
        return []

    # list of nodes visited during the random walk
    visited = [current_node_index]
    
    transition_probs = None

    for _ in range(steps):

        # identify neighbours of current node
        neighbours = np.nonzero(A[current_node_index])[1]

        # if reached an absorbing state, i.e. no neighbours, then terminate the random walk
        if neighbours.size == 0:
            #print("Reached absorbing state after", step, "steps")
            visited = list(set(visited))
            return np.array(G.nodes())[visited]
        
        # if using transition probabilities, get them
        if p:
            transition_probs = A[current_node_index].toarray()[0, neighbours]
        
        # select the index of next node to transition to
        current_node_index = np.random.choice(neighbours, p=transition_probs)

        # maintain record of the path taken by the random walk
        visited.append(current_node_index)
    
    # return unique pages visited
    visited = list(set(visited))
        
    return np.array(G.nodes())[visited]

In [7]:
import networkx as nx
import numpy as np
import pandas as pd
from utils import randomwalks as rw
from scipy.sparse import csr_matrix

## Create transition matrix

### Directed graph

In [9]:
# Get directed graph
G = nx.read_gpickle("./functional_directed_graph_uk.gpickle")

# Create array with edge weight
T = nx.adjacency_matrix(G, weight="edgeWeight").todense()
T_array = np.array(T)

# Transform edge weight into probabilities

# Normalisation
sum_of_rows = T_array.sum(axis=1)
T_probs = T_array / sum_of_rows[:, np.newaxis]

# Rows with only 0s = nan. Replace nan values with 1/Tarray.shape[0]
np.nan_to_num(T_probs, nan=1 / T_array.shape[0], copy=False)

# Convert into a transition matrix (for random walks function)
T_directed = csr_matrix(T_probs)

FileNotFoundError: [Errno 2] No such file or directory: './functional_directed_graph_uk.gpickle'

### Undirected graph
*G.to_undirected() can not be used to control what data the undirected edges get, therefore we need to create a new graph and sum the edge weights*

In [ ]:
# Create a copy of the edges with weight = 0
G_undirected = nx.Graph()
G_undirected.add_nodes_from(G.nodes(data=True))
G_undirected.add_edges_from(G.edges, edgeWeight=0)

# Sum weights for each edge
for u, v, d in G.edges(data=True):
    G_undirected[u][v]["edgeWeight"] += d["edgeWeight"]

In [ ]:
# Create array with edge weight
T_undirected = nx.adjacency_matrix(G_undirected, weight="edgeWeight").todense()
T_undirected_array = np.array(T_undirected)

# Normalisation
sum_of_rows = T_undirected_array.sum(axis=1)
T_undirected_probs = T_undirected_array / sum_of_rows[:, np.newaxis]
np.nan_to_num(T_undirected_probs, nan=1 / T_undirected_array.shape[0], copy=False)

# Convert into a transition matrix (for random walks function)
T_undirected = csr_matrix(T_undirected_probs)

## Random walks

In [ ]:
# Set the seeds from where random walks will be initialised
seeds = (
    "/browse/visas-immigration/work-visas",
    "/browse/visas-immigration/what-you-need-to-do",
    "/check-uk-visa",
    "/apply-to-come-to-the-uk",
    "/contact-ukvi-inside-outside-uk",
    "/skilled-worker-visa",
)

### Directed graph

In [ ]:
# Reformat the graph to make it compliant with existing random walk functions
# i.e. add the path to a name property and set the index to be a number
for index, data in G.nodes(data=True):
    data["properties"] = dict()
    data["properties"]["name"] = index

G = nx.convert_node_labels_to_integers(
    G, first_label=0, ordering="default", label_attribute=None
)

In [ ]:
results_directed = rw.repeat_random_walks(
    steps=100,
    repeats=100,
    T=T_directed,
    G=G,
    seed_pages=seeds,
    proba=True,
    combine="union",
    level=1,
    n_jobs=1,
)
page_scores_directed = rw.page_freq_path_freq_ranking(results_directed)

### Undirected graph

In [ ]:
# reformat the graph to make it compliant with existing random walk functions
# i.e. add the path to a name property and set the index to be a number
for index, data in G_undirected.nodes(data=True):
    data["properties"] = dict()
    data["properties"]["name"] = index

G_undirected = nx.convert_node_labels_to_integers(
    G_undirected, first_label=0, ordering="default", label_attribute=None
)

In [ ]:
results_undirected = rw.repeat_random_walks(
    steps=100,
    repeats=100,
    T=T_undirected,
    G=G_undirected,
    seed_pages=seeds,
    proba=True,
    combine="union",
    level=1,
    n_jobs=1,
)
page_scores_undirected = rw.page_freq_path_freq_ranking(results_undirected)

## Create output


In [ ]:
# Document supertypes
news_and_comms_doctypes = {
    "medical_safety_alert",
    "drug_safety_update",
    "news_article",
    "news_story",
    "press_release",
    "world_location_news_article",
    "world_news_story",
    "fatality_notice",
    "fatality_notice",
    "tax_tribunal_decision",
    "utaac_decision",
    "asylum_support_decision",
    "employment_appeal_tribunal_decision",
    "employment_tribunal_decision",
    "employment_tribunal_decision",
    "service_standard_report",
    "cma_case",
    "decision",
    "oral_statement",
    "written_statement",
    "authored_article",
    "correspondence",
    "speech",
    "government_response",
    "case_study",
}

service_doctypes = {
    "completed_transaction",
    "local_transaction",
    "form",
    "calculator",
    "smart_answer",
    "simple_smart_answer",
    "place",
    "licence",
    "step_by_step_nav",
    "transaction",
    "answer",
    "guide",
}

guidance_and_reg_doctypes = {
    "regulation",
    "detailed_guide",
    "manual",
    "manual_section",
    "guidance",
    "map",
    "calendar",
    "statutory_guidance",
    "notice",
    "international_treaty",
    "travel_advice",
    "promotional",
    "international_development_fund",
    "countryside_stewardship_grant",
    "esi_fund",
    "business_finance_support_scheme",
    "statutory_instrument",
    "hmrc_manual",
    "standard",
}

policy_and_engage_doctypes = {
    "impact_assessment",
    "policy_paper",
    "open_consultation",
    "policy_paper",
    "closed_consultation",
    "consultation_outcome",
    "policy_and_engagement",
}

research_and_stats_doctypes = {
    "dfid_research_output",
    "independent_report",
    "research",
    "statistics",
    "national_statistics",
    "statistics_announcement",
    "national_statistics_announcement",
    "official_statistics_announcement",
    "statistical_data_set",
    "official_statistics",
}

transparency_doctypes = {
    "transparency",
    "corporate_report",
    "foi_release",
    "aaib_report",
    "raib_report",
    "maib_report",
}

### Directed graph

In [ ]:
# Create a df with `pagePath`: `documentType`, `sessionHitsAll`, `entranceHit`, `exitHit`, `entranceAndExitHit`
df_dict = {
    info["properties"]["name"]: [
        info["documentType"],
        info["sessionHitsAll"],
        info["entranceHit"],
        info["exitHit"],
        info["entranceAndExitHit"],
        info["sessionHits"],
    ]
    for node, info in G.nodes(data=True)
}
df_dict = {
    k: v for (k, v) in df_dict.items() if k in page_scores_directed["pagePath"].tolist()
}
df_info = (
    pd.DataFrame.from_dict(
        df_dict,
        orient="index",
        columns=[
            "documentType",
            "sessionHitsAll",
            "entranceHit",
            "exitHit",
            "entranceAndExitHit",
            "sessionHits",
        ],
    )
    .rename_axis("pagePath")
    .reset_index()
)

In [ ]:
# Create a df with document supertypes
document_type_dict = dict.fromkeys(list(set(df_info["documentType"])))

for docType, docSupertype in document_type_dict.items():
    if docType in news_and_comms_doctypes:
        document_type_dict[docType] = "news and communication"

    elif docType in service_doctypes:
        document_type_dict[docType] = "services"

    elif docType in guidance_and_reg_doctypes:
        document_type_dict[docType] = "guidance and regulation"

    elif docType in policy_and_engage_doctypes:
        document_type_dict[docType] = "policy and engagement"

    elif docType in research_and_stats_doctypes:
        document_type_dict[docType] = "research and statistics"

    elif docType in transparency_doctypes:
        document_type_dict[docType] = "transparency"

    else:
        document_type_dict[docType] = "other"

df_docSuper = pd.DataFrame(
    document_type_dict.items(), columns=["documentType", "documentSupertype"]
)

In [ ]:
# Merge dfs
df_merged = pd.merge(page_scores_directed, df_info, on="pagePath")
df_merged = pd.merge(df_merged, df_docSuper, how="left")

# Reoder and rename df columns
df_merged = df_merged[
    [
        "pagePath",
        "documentType",
        "documentSupertype",
        "sessionHitsAll",
        "entranceHit",
        "exitHit",
        "entranceAndExitHit",
        "sessionHits",
        "tfdf_max",
    ]
]
df_merged = df_merged.rename(
    columns={
        "pagePath": "page path",
        "documentType": "document type",
        "documentSupertype": "document supertype",
        "sessionHitsAll": "number of sessions that visit this page",
        "entranceHit": "number of sessions where this page is an entrance hit",
        "exitHit": "number of sessions where this page is an exit hit",
        "entranceAndExitHit": "number of sessions where this page is both an entrance and exit hit",
        "sessionHits": "all sessions that visit this page, regardless of the session visiting a seed page",
        "tfdf_max": "how frequent the page occurs in the whole user journey",
    }
)

# Save df
df_merged.to_csv("../../outputs/pages_ranked_directed_uk.csv", index=False)

### Undirected graph

In [ ]:
# Create a df with `pagePath`: `documentType`, `sessionHitsAll`, `entranceHit`, `exitHit`, `entranceAndExitHit`
df_dict = {
    info["properties"]["name"]: [
        info["documentType"],
        info["sessionHitsAll"],
        info["entranceHit"],
        info["exitHit"],
        info["entranceAndExitHit"],
        info["sessionHits"],
    ]
    for node, info in G_undirected.nodes(data=True)
}
df_dict = {
    k: v
    for (k, v) in df_dict.items()
    if k in page_scores_undirected["pagePath"].tolist()
}
df_info = (
    pd.DataFrame.from_dict(
        df_dict,
        orient="index",
        columns=[
            "documentType",
            "sessionHitsAll",
            "entranceHit",
            "exitHit",
            "entranceAndExitHit",
            "sessionHits",
        ],
    )
    .rename_axis("pagePath")
    .reset_index()
)

In [ ]:
# Create a df with document supertypes
document_type_dict = dict.fromkeys(list(set(df_info["documentType"])))

for docType, docSupertype in document_type_dict.items():
    if docType in news_and_comms_doctypes:
        document_type_dict[docType] = "news and communication"

    elif docType in service_doctypes:
        document_type_dict[docType] = "services"

    elif docType in guidance_and_reg_doctypes:
        document_type_dict[docType] = "guidance and regulation"

    elif docType in policy_and_engage_doctypes:
        document_type_dict[docType] = "policy and engagement"

    elif docType in research_and_stats_doctypes:
        document_type_dict[docType] = "research and statistics"

    elif docType in transparency_doctypes:
        document_type_dict[docType] = "transparency"

    else:
        document_type_dict[docType] = "other"

df_docSuper = pd.DataFrame(
    document_type_dict.items(), columns=["documentType", "documentSupertype"]
)

In [ ]:
# Merge dfs
df_merged = pd.merge(page_scores_undirected, df_info, on="pagePath")
df_merged = pd.merge(df_merged, df_docSuper, how="left")

# Reoder and rename df columns
df_merged = df_merged[
    [
        "pagePath",
        "documentType",
        "documentSupertype",
        "sessionHitsAll",
        "entranceHit",
        "exitHit",
        "entranceAndExitHit",
        "sessionHits",
        "tfdf_max",
    ]
]
df_merged = df_merged.rename(
    columns={
        "pagePath": "page path",
        "documentType": "document type",
        "documentSupertype": "document supertype",
        "sessionHitsAll": "number of sessions that visit this page",
        "entranceHit": "number of sessions where this page is an entrance hit",
        "exitHit": "number of sessions where this page is an exit hit",
        "entranceAndExitHit": "number of sessions where this page is both an entrance and exit hit",
        "sessionHits": "all sessions that visit this page, regardless of the session visiting a seed page",
        "tfdf_max": "how frequent the page occurs in the whole user journey",
    }
)

# Save df
df_merged.to_csv("../../outputs/pages_ranked_undirected_uk.csv", index=False)